<a href="https://www.kaggle.com/code/pogscafe/hotshot-xl?scriptVersionId=146057112" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
BASEPATH = '/kaggle/working'

%cd $BASEPATH
!git clone https://github.com/wandaweb/Hotshot-XL
HOTSHOT_PATH = BASEPATH + '/Hotshot-XL'
%cd $HOTSHOT_PATH
!pip install -r requirements.txt

import os
os.makedirs('lora', exist_ok=True)
os.makedirs('input', exist_ok=True)
os.makedirs('output', exist_ok=True)

!apt install -y imagemagick

In [ ]:
%cd $HOTSHOT_PATH

#Parameters

batch_size = 1 # @param {type:"slider", min:1, max:50, step:1}
positive_prompt = 'teddy bear dancing in a cyberpunk city square, dramatic lighting, dark, slow motion, colorful, octane render, centered, looking at viewer' #@param {type:"string"}
negative_prompt = 'blurry, fog' #@param {type:"string"}
seed = -1 #@param {type:"integer"}
width = 512 #@param {type:"integer"}
height = 512 #@param {type:"integer"}
upscale = "2" #@param [0, 2, 4]
video_frames = 8 #@param {type:"integer"}
video_duration = 1000 #@param {type:"integer"}
steps = "40" #@param {type:"string"}
lora_file_name = '' #@param {type:"string"}
file_prefix = 'out_' #@param {type:"string"}
file_type = '.mp4' #@param [".mp4", ".gif"]
scheduler='EulerDiscreteScheduler' #@param ["EulerAncestralDiscreteScheduler", "EulerDiscreteScheduler"]

# Gif to video
gif_name = '' #@param {type:"string"}
control_type = 'depth' #@param ["depth", "canny"]
controlnet_conditioning_scale = 0.8 #@param {type:"number"}
control_guidance_start = 0.0 #@param {type:"number"}
control_guidance_end = 1.0 #@param {type:"number"}

# Resolutions quick reference

# | Width x Height | Ratio| Orientation |
# | --- | --- | --- |
# | 512 x 512 | 1 : 1 | Square |
# | 608 x 416 | 19 : 13 | Horizontal
# | 416 x 608 | 13 : 19 | Vertical
# | 672 x 384 | 7 : 4 | Horizontal
# | 384 x 672 | 4 : 7 | Vertical
# | 768 x 320 | 12 : 5 | Horizontal
# | 320 x 768 | 5 : 12 | Vertical

import datetime, random, os
from IPython.display import display, Image, HTML
from base64 import b64encode

videos = []

if seed == None or seed == -1:
  seed = random.randint(0, 18446744073709519871)

for i in range(batch_size):
  file_name=f'{file_prefix}{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}{file_type}'
  data_name=f'{file_prefix}{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.txt'
  # Create the command
  cmd = f'''python inference.py \
   --prompt="{positive_prompt}" \
   --negative_prompt="{negative_prompt}" \
   --steps={steps} \
   --seed={seed} \
   --width={width} \
   --height={height} \
   --video_length={video_frames} \
   --video_duration={video_duration} \
   --scheduler="{scheduler}" \
   '''
  if upscale == '2' or upscale == '4':
    cmd += f'--upscale={upscale} \\'
  if not lora_file_name == '':
    print('using lora')
    cmd += f'--lora="lora/{lora_file_name}" \\'
  if not gif_name == '':
    cmd += f'''--gif=input/{gif_name} \
      --control_type={control_type} \
      --controlnet_conditioning_scale={controlnet_conditioning_scale} \
      --control_guidance_start={control_guidance_start} \
      --control_guidance_end={control_guidance_end} \
      '''

  cmd += f'--output="output/{file_name}"'

  # Execute command
  print(cmd)
  get_ipython().system(cmd);

  # Write command to file
  f = open('output/'+data_name, 'w')
  f.write(cmd)
  f.close()

  seed = random.randint(0, 18446744073709519871)
  videos.append('output/' + file_name)

# Show the results
print(str(videos))
result = ''
if file_type == '.gif':
  for v in videos:
    display(Image(filename=v, retina=True))
else:
  for v in videos:
      mp4 = open(v,'rb').read()
      data_url = 'data:video/mp4;base64,' + b64encode(mp4).decode()
      result += ('''
      <video width=400 controls loop>
        <source src="%s" type="video/mp4">
      </video>
      ''' % data_url)
HTML(result)

In [ ]:
#@title Download a LoRA

%cd $HOTSHOT_PATH
%cd lora

import requests

url = 'https://civitai.com/api/download/models/163063'  #@param {type:"string"}
lora_file_name = 'woolify.safetensors' #@param {type:"string"}

r = requests.get(url, allow_redirects=True)
with open(lora_file_name, 'wb') as f:
    f.write(r.content)

print('Downloaded ' + lora_file_name)

In [ ]:
#@title Download gif

%cd $HOTSHOT_PATH
%cd input

import requests
from PIL import Image
from io import BytesIO

url='https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExZGFiY2M1bnl0bjVrNjdxYWViZTZsZHZwOXNncmNjbGhuam9lM3F0ZSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/0VKbVY3N0VW0ZegLbe/giphy.gif' #@param {type:"string"}
gif_name = 'bunny.gif' #@param {type:"string"}

r = requests.get(url, allow_redirects=True)
with open(gif_name, 'wb') as f:
    f.write(r.content)

# Show the frame count
im = Image.open(gif_name)
print(f'{gif_name} has {im.n_frames} frames')

In [ ]:
#@title Download webp and convert to gif

%cd $HOTSHOT_PATH
%cd input

import requests
from PIL import Image
from io import BytesIO

url='' #@param {type:"string"}
gif_name = '' #@param {type:"string"}

r = requests.get(url, allow_redirects=True)
with open('temp.webp', 'wb') as f:
    f.write(r.content)

cmd = f'convert temp.webp {gif_name}'
get_ipython().system(cmd)

# Show the frame count
im = Image.open(gif_name)
print(f'{gif_name} has {im.n_frames} frames')